In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re #Regular Expressions
import string #importing string functions
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.stem.porter import PorterStemmer

In [2]:
from tensorflow.keras.layers import Embedding,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional,GRU,MaxPooling1D,Conv1D
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import *
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, roc_auc_score,roc_curve,auc

In [3]:
df= pd.read_csv('Market_Basket_Optimisation.csv',header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.values[0]

array(['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes',
       'whole weat flour', 'yams', 'cottage cheese', 'energy drink',
       'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad',
       'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie',
       'spinach', 'olive oil'], dtype=object)

In [5]:
print(df.shape)

(7501, 20)


In [6]:
transactions1 = []
for i,row in df.iterrows():
   transactions1.append(row.values.astype(str).tolist())
transactions1[0:5]

[['shrimp',
  'almonds',
  'avocado',
  'vegetables mix',
  'green grapes',
  'whole weat flour',
  'yams',
  'cottage cheese',
  'energy drink',
  'tomato juice',
  'low fat yogurt',
  'green tea',
  'honey',
  'salad',
  'mineral water',
  'salmon',
  'antioxydant juice',
  'frozen smoothie',
  'spinach',
  'olive oil'],
 ['burgers',
  'meatballs',
  'eggs',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['chutney',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['turkey',
  'avocado',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['mineral water',
  'milk',
  'energy bar',
  'whole wheat rice',
  'green tea',
  'nan',
  'nan',
  'nan',
 

In [7]:
# Convert your transaction data into a transaction matrix
# Here, `transactions` is a list of lists, where each inner list represents a transaction
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit_transform(transactions1)
transaction_df = pd.DataFrame(te_ary, columns=te.columns_)
transaction_df

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,True,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7497,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7498,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7499,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
!pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5955 sha256=7dde4f73a4286f773842ab7944bc81dd8aba2cd28d96146c34dbc00465f9920b
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [9]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from itertools import product

# Define the dataset or transactions

# Grid search parameters
min_support = [0.003, 0.002, 0.004]
min_confidence = [0.1, 0.2, 0.3]
max_len = [2, 3, 4]

# Perform grid search
best_rules = None
best_params = None
best_support = 0

for params in product(min_support, min_confidence, max_len):
    # Run Apriori algorithm with current parameters
    frequent_itemsets = apriori(transaction_df, min_support=params[0], max_len=params[2])
    rules = association_rules(frequent_itemsets, min_threshold=params[1])

    # Check if current rules are better than previous best rules
    if len(rules) > 0 and rules['support'].mean() > best_support:
        best_rules = rules
        best_params = params
        best_support = rules['support'].mean()

print("Best parameters:", best_params)
print("Best support:", best_support)
print("Best rules:")
print(best_rules)


Best parameters: (0.004, 0.3, 2)
Best support: 0.025981493065881815
Best rules:
    antecedents consequents  antecedent support  consequent support   support  \
0           (1)        (37)            0.020397            0.179709  0.006532   
1           (1)        (72)            0.020397            0.238368  0.007599   
2           (1)        (77)            0.020397            0.999867  0.020264   
3           (2)        (77)            0.008932            0.999867  0.008799   
4           (3)        (77)            0.004666            0.999867  0.004666   
..          ...         ...                 ...                 ...       ...   
190        (86)       (101)            0.026530            0.174110  0.009865   
191        (90)       (101)            0.028130            0.174110  0.010265   
192        (93)       (101)            0.042528            0.174110  0.013465   
193       (108)       (101)            0.014131            0.174110  0.006266   
194       (109)       (101)  

In [10]:
from apyori import apriori
rules = apriori(transactions = transactions1, min_support = 0.004, min_confidence =0.3, min_lift = 2, min_length = 2, max_length = 2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
results = list(rules)
results

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[RelationRecord(items=frozenset({'french fries', 'body spray'}), support=0.004266097853619517, ordered_statistics=[OrderedStatistic(items_base=frozenset({'body spray'}), items_add=frozenset({'french fries'}), confidence=0.37209302325581395, lift=2.17712150346479)]),
 RelationRecord(items=frozenset({'chocolate', 'tomato sauce'}), support=0.005065991201173177, ordered_statistics=[OrderedStatistic(items_base=frozenset({'tomato sauce'}), items_add=frozenset({'chocolate'}), confidence=0.3584905660377358, lift=2.1879883936932925)]),
 RelationRecord(items=frozenset({'eggs', 'cider'}), support=0.004266097853619517, ordered_statistics=[OrderedStatistic(items_base=frozenset({'cider'}), items_add=frozenset({'eggs'}), confidence=0.4050632911392405, lift=2.2539909101153137)]),
 RelationRecord(items=frozenset({'escalope', 'mushroom cream sauce'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), conf

In [12]:
result=results[0]
result

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


RelationRecord(items=frozenset({'french fries', 'body spray'}), support=0.004266097853619517, ordered_statistics=[OrderedStatistic(items_base=frozenset({'body spray'}), items_add=frozenset({'french fries'}), confidence=0.37209302325581395, lift=2.17712150346479)])

In [13]:
result[2]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[OrderedStatistic(items_base=frozenset({'body spray'}), items_add=frozenset({'french fries'}), confidence=0.37209302325581395, lift=2.17712150346479)]

In [14]:
result[2][0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


OrderedStatistic(items_base=frozenset({'body spray'}), items_add=frozenset({'french fries'}), confidence=0.37209302325581395, lift=2.17712150346479)

In [15]:
result[2][0][0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


frozenset({'body spray'})

In [16]:
tuple(result[2][0][0])[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'body spray'

In [17]:
lhs1=[]
for result in results:
   lhs1.append(tuple(result[2][0][0])[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
lhs1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['body spray',
 'tomato sauce',
 'cider',
 'mushroom cream sauce',
 'pasta',
 'extra dark chocolate',
 'herb & pepper',
 'ground beef',
 'tomato sauce',
 'soup',
 'whole wheat pasta',
 'nonfat milk',
 'pasta',
 'pepper',
 'red wine',
 'tomato sauce']

In [19]:
supprt=result[1]
supprt

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.006265831222503666

In [20]:
result[2][0][2]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.4433962264150943

In [21]:
def total(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
reslult_df=total(results)
reslult_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('body spray',
  'french fries',
  0.004266097853619517,
  0.37209302325581395,
  2.17712150346479),
 ('tomato sauce',
  'chocolate',
  0.005065991201173177,
  0.3584905660377358,
  2.1879883936932925),
 ('cider',
  'eggs',
  0.004266097853619517,
  0.4050632911392405,
  2.2539909101153137),
 ('mushroom cream sauce',
  'escalope',
  0.005732568990801226,
  0.3006993006993007,
  3.790832696715049),
 ('pasta',
  'escalope',
  0.005865884548726837,
  0.3728813559322034,
  4.700811850163794),
 ('extra dark chocolate',
  'mineral water',
  0.005732568990801226,
  0.47777777777777775,
  2.0043686303753416),
 ('herb & pepper',
  'ground beef',
  0.015997866951073192,
  0.3234501347708895,
  3.2919938411349285),
 ('ground beef',
  'spaghetti',
  0.03919477403012932,
  0.3989145183175034,
  2.291162176033379),
 ('tomato sauce',
  'ground beef',
  0.005332622317024397,
  0.3773584905660377,
  3.840659481324083),
 ('soup',
  'milk',
  0.015197973603519531,
  0.3007915567282322,
  2.3212319619531

In [22]:
resultsinDataFrame = pd.DataFrame(reslult_df, columns = ['Left Side', 'Right Side', 'Support', 'Confidence', 'Lift'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
resultsinDataFrame

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Left Side,Right Side,Support,Confidence,Lift
0,body spray,french fries,0.004266,0.372093,2.177122
1,tomato sauce,chocolate,0.005066,0.358491,2.187988
2,cider,eggs,0.004266,0.405063,2.253991
3,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
4,pasta,escalope,0.005866,0.372881,4.700812
5,extra dark chocolate,mineral water,0.005733,0.477778,2.004369
6,herb & pepper,ground beef,0.015998,0.323450,3.291994
7,ground beef,spaghetti,0.039195,0.398915,2.291162
8,tomato sauce,ground beef,0.005333,0.377358,3.840659
9,soup,milk,0.015198,0.300792,2.321232


In [24]:
resultsinDataFrame.nlargest(n = 9, columns = 'Lift')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Left Side,Right Side,Support,Confidence,Lift
4,pasta,escalope,0.005866,0.372881,4.700812
12,pasta,shrimp,0.005066,0.322034,4.506672
8,tomato sauce,ground beef,0.005333,0.377358,3.840659
3,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
6,herb & pepper,ground beef,0.015998,0.323450,3.291994
10,whole wheat pasta,milk,0.009865,0.334842,2.583999
15,tomato sauce,spaghetti,0.006266,0.443396,2.546642
9,soup,milk,0.015198,0.300792,2.321232
7,ground beef,spaghetti,0.039195,0.398915,2.291162
